In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, precision_recall_curve, auc,roc_auc_score,average_precision_score

from sklearn.preprocessing import StandardScaler

In [2]:
train_df = pd.read_csv('../dataset/train.csv')
val_df = pd.read_csv('../dataset/val.csv')
test_df = pd.read_csv('../dataset/test.csv')

In [3]:
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
col=[col for col in val_df.columns if col not in ['ID','Class']]

for i in col:
    sc=RobustScaler()
    train_df[i]=sc.fit_transform(train_df[i].values.reshape(-1,1))
    val_df[i]=sc.transform(val_df[i].values.reshape(-1,1))
    test_df[i]=sc.transform(test_df[i].values.reshape(-1,1))

In [4]:
# train_df['cls'] = 'train'
# val_df['cls'] = 'val'
# test_df['cls'] = 'test'

In [5]:
# df = pd.concat([train_df, val_df.drop(columns=['Class'], axis=1), test_df], axis=0)
# df.reset_index(drop=True, inplace=True)
# df.shape

In [6]:
u, s, v = np.linalg.svd(train_df.drop(columns=['ID'], axis=1).to_numpy(), full_matrices=False)

In [7]:
print(u.shape)
print(v.shape)
s

(113842, 30)
(30, 30)


array([1278.31587653,  923.02866545,  831.5758778 ,  743.96379981,
        719.93720733,  659.6348925 ,  578.43414006,  392.63750934,
        383.99020809,  369.01430811,  361.70525652,  346.8854685 ,
        331.55525639,  312.36230932,  303.3202389 ,  300.88841083,
        300.45109438,  298.92743991,  296.50955018,  287.711925  ,
        287.22094797,  266.57576546,  259.6877166 ,  257.87966741,
        256.23595985,  252.12659428,  232.53828975,  226.48695432,
        124.41829001,  116.51372244])

In [8]:
s[:1] = 0
s

array([  0.        , 923.02866545, 831.5758778 , 743.96379981,
       719.93720733, 659.6348925 , 578.43414006, 392.63750934,
       383.99020809, 369.01430811, 361.70525652, 346.8854685 ,
       331.55525639, 312.36230932, 303.3202389 , 300.88841083,
       300.45109438, 298.92743991, 296.50955018, 287.711925  ,
       287.22094797, 266.57576546, 259.6877166 , 257.87966741,
       256.23595985, 252.12659428, 232.53828975, 226.48695432,
       124.41829001, 116.51372244])

In [9]:
train = pd.DataFrame(u @ np.diag(s) @ v)
train.shape

(113842, 30)

In [10]:
u, s, v = np.linalg.svd(val_df.drop(columns=['ID', 'Class'], axis=1).to_numpy(), full_matrices=False)
s


array([635.84907593, 412.42398601, 396.5354038 , 347.8739693 ,
       325.09660704, 304.77998353, 273.11755181, 192.58933042,
       186.1350304 , 182.73312737, 181.14346537, 173.63576211,
       167.52087355, 156.06270339, 150.509728  , 149.55735871,
       149.49693884, 148.05117585, 146.28984452, 145.39915159,
       143.68538213, 138.70931491, 132.04636009, 129.00092592,
       128.2035067 , 125.51971999, 116.6876217 , 113.64423059,
        62.22020122,  51.48874989])

In [11]:
s[:1] = 0
val = pd.DataFrame(u @ np.diag(s) @ v)

In [12]:
val.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.163530,0.728568,0.441554,-0.124136,0.411199,0.032112,0.560521,0.081138,-0.553856,-0.282432,...,-0.509154,-0.602729,-0.357694,-0.536623,-0.132761,0.260169,1.505955,0.549246,-0.109937,-0.990470
1,0.423012,0.182194,-0.186341,1.334369,0.904835,1.682646,0.064255,0.928759,-0.920229,0.823776,...,0.421777,0.368255,-0.118589,-1.779419,0.556197,0.446009,0.090972,-0.196374,0.200794,-0.991059
2,0.507457,0.368305,0.006360,1.639636,-0.027025,-0.660772,0.437087,-0.487218,-0.681585,0.858958,...,0.097536,-0.119429,-0.206267,0.819855,0.919925,0.178964,-0.336766,0.038736,0.145722,-0.991109
3,0.416752,-0.250357,0.113358,0.349913,0.245779,1.629816,-0.432335,1.115750,0.225986,-0.132487,...,-0.038294,0.068814,0.359892,-1.300997,0.101290,0.757833,0.217351,-0.131350,0.165744,-0.991920
4,-1.265418,2.107212,0.751219,-1.537748,0.430197,-0.590337,1.644496,-3.333499,3.291489,6.250170,...,-2.116974,-0.153039,-0.405217,0.603865,0.191438,0.894222,-1.518527,-8.527418,-1.985781,-1.000961


In [13]:
from sklearn.ensemble import IsolationForest

IF=IsolationForest(n_estimators=100,n_jobs=-1,contamination=0.001,random_state=7,max_samples=0.25)
# IF=IsolationForest(n_estimators=100,n_jobs=-1,contamination='auto',random_state=7,max_samples=0.25)
IF.fit(train)


IsolationForest(contamination=0.001, max_samples=0.25, n_jobs=-1,
                random_state=7)

In [14]:
# X=val.drop(labels=['ID','Class', 'cls'],axis=1)
# print(X.shape)
val['If_predict']=IF.predict(val)
val['If_predict'].replace({1:0,-1:1},inplace=True)
val['Class'] = val_df['Class']

In [15]:
val['If_predict'].value_counts()

0    28438
1       24
Name: If_predict, dtype: int64

In [16]:
val['Class'].value_counts()

0    28432
1       30
Name: Class, dtype: int64

In [17]:
from sklearn.metrics import f1_score
y_true = val.Class.values
y_pred = val.If_predict.values
f1_score(y_true, y_pred, average='macro')

0.7219584627708419

In [18]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.50      0.40      0.44        30

    accuracy                           1.00     28462
   macro avg       0.75      0.70      0.72     28462
weighted avg       1.00      1.00      1.00     28462

